In [127]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, Normalizer, OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix,ConfusionMatrixDisplay
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessing/test1.csv
/kaggle/input/preprocessing/training1.csv
/kaggle/input/preprocessing/training_set_labels.csv
/kaggle/input/preprocessing/submission_format.csv
/kaggle/input/covidv/training.csv
/kaggle/input/covidv/training_set_labels.csv
/kaggle/input/covidv/submission_format.csv
/kaggle/input/covidv/testing.csv


In [128]:
training = pd.read_csv('/kaggle/input/preprocessing/training1.csv')
test = pd.read_csv('/kaggle/input/preprocessing/test1.csv')
labels = pd.read_csv('/kaggle/input/covidv/training_set_labels.csv')

In [129]:
pd.set_option('display.max_columns', None)

In [130]:
training['behavioral_hygene'] = training['behavioral_face_mask'] + training['behavioral_wash_hands']
test['behavioral_hygene'] = test['behavioral_face_mask'] + test['behavioral_wash_hands']

In [131]:
training['ratio'] = training['opinion_seas_sick_from_vacc'] / training['opinion_seas_risk']
test['ratio'] = test['opinion_seas_sick_from_vacc'] / test['opinion_seas_risk']

In [132]:
training.drop(columns = ['opinion_seas_sick_from_vacc','opinion_seas_risk','behavioral_face_mask','behavioral_wash_hands'], inplace=True)
test.drop(columns = ['opinion_seas_sick_from_vacc','opinion_seas_risk','behavioral_face_mask','behavioral_wash_hands'], inplace=True)

In [133]:
training.head()

,xyz_concern,xyz_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,opinion_xyz_vacc_effective,opinion_xyz_risk,opinion_xyz_sick_from_vacc,opinion_seas_vacc_effective,sex,rent_or_own,household_adults,household_children,edu_1.0,edu_2.0,edu_3.0,edu_4.0,ag_0,ag_1,ag_2,inc_1.0,inc_2.0,inc_3.0,wed_1.0,wed_2.0,emp_1.0,emp_2.0,d_Black,d_Hispanic,d_Other or Multiple,d_White,town_atmpeygn,town_bhuqouqj,town_dqpwygqj,town_fpwskwrf,town_kbazzjca,town_lrircsnp,town_lzgpxyit,town_mlyzmhmf,town_oxchjgsf,town_qufhixun,"region_MSA, Not Principle City","region_MSA, Principle City",region_Non-MSA,behavioral_hygene,ratio
0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,2.0,2.0,1,1,0.0,0.0,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0.0,2.00
1,3.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,4.0,4.0,0,0,0.0,0.0,0,1,0,0,0,1,0,1,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1.0,2.00
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0,1.0,4.0,0,1,2.0,0.0,0,0,0,1,1,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0.0,2.00
3,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,3.0,3.0,5.0,5.0,1,0,0.0,0.0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,1.0,0.25
4,2.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,2.0,3.0,1,1,1.0,0.0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1.0,4.00


In [134]:
training.columns

Index(['xyz_concern', 'xyz_knowledge', 'behavioral_antiviral_meds',
       'behavioral_avoidance', 'behavioral_large_gatherings',
       'behavioral_outside_home', 'behavioral_touch_face', 'doctor_recc_xyz',
       'doctor_recc_seasonal', 'chronic_med_condition', 'child_under_6_months',
       'health_worker', 'opinion_xyz_vacc_effective', 'opinion_xyz_risk',
       'opinion_xyz_sick_from_vacc', 'opinion_seas_vacc_effective', 'sex',
       'rent_or_own', 'household_adults', 'household_children', 'edu_1.0',
       'edu_2.0', 'edu_3.0', 'edu_4.0', 'ag_0', 'ag_1', 'ag_2', 'inc_1.0',
       'inc_2.0', 'inc_3.0', 'wed_1.0', 'wed_2.0', 'emp_1.0', 'emp_2.0',
       'd_Black', 'd_Hispanic', 'd_Other or Multiple', 'd_White',
       'town_atmpeygn', 'town_bhuqouqj', 'town_dqpwygqj', 'town_fpwskwrf',
       'town_kbazzjca', 'town_lrircsnp', 'town_lzgpxyit', 'town_mlyzmhmf',
       'town_oxchjgsf', 'town_qufhixun', 'region_MSA, Not Principle  City',
       'region_MSA, Principle City', 'region_Non

In [135]:
train = training[['behavioral_hygene','xyz_concern','xyz_knowledge', 'behavioral_antiviral_meds','behavioral_avoidance',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_xyz', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'opinion_xyz_vacc_effective', 'opinion_xyz_risk', 'wed_1.0', 'wed_2.0',
       'opinion_xyz_sick_from_vacc', 'opinion_seas_vacc_effective',
       'ratio','emp_1.0', 'emp_2.0','edu_2.0', 'edu_3.0', 'edu_4.0', 'ag_0', 'ag_1', 'ag_2', 'inc_1.0',
       'inc_2.0', 'inc_3.0','sex']].copy()

In [136]:
train['meds_avoidance'] = train['behavioral_antiviral_meds'] * train['behavioral_avoidance']
train.drop(columns=['behavioral_antiviral_meds', 'behavioral_avoidance'], inplace=True)


In [137]:
train.head()

,behavioral_hygene,xyz_concern,xyz_knowledge,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_xyz,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,opinion_xyz_vacc_effective,opinion_xyz_risk,wed_1.0,wed_2.0,opinion_xyz_sick_from_vacc,opinion_seas_vacc_effective,ratio,emp_1.0,emp_2.0,edu_2.0,edu_3.0,edu_4.0,ag_0,ag_1,ag_2,inc_1.0,inc_2.0,inc_3.0,sex,meds_avoidance
0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,1,0,2.0,2.0,2.00,0,1,0,0,0,0,1,0,1,0,0,1,0.0
1,1.0,3.0,2.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,1,0,4.0,4.0,2.00,1,0,1,0,0,0,1,0,1,0,0,0,0.0
2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,1.0,1,0,1.0,4.0,2.00,1,0,0,0,1,1,0,0,0,1,0,0,0.0
3,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,3.0,3.0,1,0,5.0,5.0,0.25,0,1,1,0,0,0,0,1,1,0,0,1,0.0
4,1.0,2.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0,1,2.0,3.0,4.00,1,0,0,1,0,0,1,0,0,1,0,1,0.0


In [138]:
labels.head()
y1 = labels['xyz_vaccine']
y2 = labels['seasonal_vaccine']

In [139]:
std1 = StandardScaler()
x = std1.fit_transform(train)

In [140]:
std11 = StandardScaler()
X = std11.fit_transform(training)

In [141]:
std2 = StandardScaler()
y = std2.fit_transform(test)


In [142]:
X_train, X_test, y1_train, y1_test = train_test_split(X,y1, test_size = 0.2)
x_train,x_test,y2_train,y2_test = train_test_split(x,y2, test_size = 0.2)

In [143]:
lg1 = LogisticRegression(C= 0.1, max_iter= 100,solver='lbfgs')
lg1.fit(X_train,y1_train)

LogisticRegression(C=0.1)

In [144]:
y1_pred = lg1.predict(X_test)
roc_auc_score(y1_test,y1_pred)

0.6932105353392483

In [145]:
lg2 = LogisticRegression(C = 1, max_iter=150,multi_class = 'ovr')
lg2.fit(x_train,y2_train)


LogisticRegression(C=1, max_iter=150, multi_class='ovr')

In [146]:
y2_pred = lg2.predict(x_test)
roc_auc_score(y2_test,y2_pred)

0.764466093564239

In [147]:
first = roc_auc_score(y1_test,y1_pred,average='macro')
first

0.6932105353392483

In [148]:
second = roc_auc_score(y2_test,y2_pred,average='macro')
second

0.764466093564239

In [149]:
(first + second) / 2 

0.7288383144517436

In [150]:
xyz_vaccine1 = lg1.predict_proba(test)
prob1 = xyz_vaccine1[:,1]
prob1 = np.ceil(prob1 * 100) / 100

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [151]:
test1 = test[['behavioral_hygene','xyz_concern','xyz_knowledge', 'behavioral_antiviral_meds','behavioral_avoidance',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_xyz', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'opinion_xyz_vacc_effective', 'opinion_xyz_risk', 'wed_1.0', 'wed_2.0',
       'opinion_xyz_sick_from_vacc', 'opinion_seas_vacc_effective',
       'ratio','emp_1.0', 'emp_2.0','edu_2.0', 'edu_3.0', 'edu_4.0', 'ag_0', 'ag_1', 'ag_2', 'inc_1.0',
       'inc_2.0', 'inc_3.0','sex']].copy()

In [152]:
test1['meds_avoidance'] = test1['behavioral_antiviral_meds'] * test1['behavioral_avoidance']
test1.drop(columns=['behavioral_antiviral_meds', 'behavioral_avoidance'], inplace=True)

In [153]:
seasonal_vaccine = lg2.predict_proba(test1)
seasonal_vaccine = np.ceil(seasonal_vaccine * 100) / 100
prob2 = seasonal_vaccine[:,1]
prob2

/opt/conda/lib/python3.10/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


array([0.97, 0.88, 0.99, ..., 0.95, 0.96, 0.97])

In [154]:
combined = np.column_stack((prob1,prob2))
custom_index = np.arange(26707, 26707 + len(xyz_vaccine1))
df = pd.DataFrame(combined, columns=['xyz_vaccine', 'seasonal_vaccine'], index=custom_index)

In [155]:
df

,xyz_vaccine,seasonal_vaccine
26707,0.88,0.97
26708,0.79,0.88
26709,0.98,0.99
26710,0.95,0.99
26711,0.92,0.97
...,...,...
53410,0.93,0.98
53411,0.79,0.97
53412,0.92,0.95
53413,0.77,0.96


In [96]:
df.to_csv('/kaggle/working/submission.csv')